In [1]:
from sklearn.model_selection import train_test_split

import cv2
import pandas as pd
import numpy as np
import json

In [2]:
from keras.layers.core import Dense, Flatten, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.advanced_activations import ELU
from keras.layers import Lambda
from keras.models import Sequential
from keras.optimizers import Adam

Using TensorFlow backend.


In [3]:
def random_Y(yuv_image):
    random_y = 0.5 + np.random.uniform()
    yuv_image[:,:,0] = yuv_image[:,:,0]*random_y
    return yuv_image

def horizontal_flip(image, angle):
    flipped_image = cv2.flip(image, 1)
    flipped_angle = angle*(-1)
    return flipped_image, flipped_angle

In [4]:
log = pd.read_csv('./data/driving_log.csv')
# X_train_paths = pd.concat([log['center'], log['left'], log['right']])
X_train_paths = log['center']
# y_train = pd.concat([log['steering'], log['steering']  + 0.08, log['steering'] - 0.08])
y_train = log['steering']
X_train_paths, X_valid_paths, y_train, y_valid = train_test_split(X_train_paths,
                                                                 y_train,
                                                                 test_size = 0.2)

batch_size = 100

In [5]:
# 2 more image for every image.
def augment_and_process(path, angle):
    images = np.empty([3, 66, 200, 3])
    angles = np.empty([3, 1])
    
    datapath = './data/' + path
    datapath = datapath.replace(" ", "")
    
    image = cv2.imread(datapath)
    image = image[12:148, 0:320]
    image = cv2.resize(image, (200, 66))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    images[0] = image
    images[1] = random_Y(image)
    flipped_image, flipped_angle = horizontal_flip(image, angle)
    images[2] = flipped_image
    angles[0] = angle
    angles[1] = angle
    angles[2] = flipped_angle
    return images, angles
    

In [6]:
def generator(X, Y):
    while 1:
        for batch in range(len(X_train_paths)//batch_size + 2):
            batch = batch*batch_size
            feature_batch = X[batch:batch + batch_size]
            angle_batch = Y[batch:batch + batch_size]
            images = []
            angles = []
            for (x, y) in zip(feature_batch, angle_batch):
                image, angle = augment_and_process(x, y)
                images.append(image)
                angles.append(angle)
            try:
                images = np.vstack(images)
                angles = np.vstack(angles)
            except:
                continue
                
            yield images, angles

In [7]:
def the_model():
    model = Sequential()
    ch, row, col = 3, 66, 200
    model.add(Lambda(lambda x: x/127.5 - 1.0,
            input_shape=(row, col, ch)))
    model.add(Convolution2D(24, 5, 5, subsample=(2, 2), border_mode='same', init='normal', input_shape=(row, col, ch)))
    model.add(ELU())
    
    model.add(Convolution2D(36, 5, 5, subsample=(2, 2), init='normal'))
    model.add(Dropout(0.5))
    model.add(ELU())
    
    model.add(Convolution2D(48, 5, 5, subsample=(2, 2), init='normal'))
    model.add(ELU())
    
    model.add(Convolution2D(64, 3, 3, subsample=(1, 1), init='normal'))
    model.add(Dropout(0.5))
    model.add(ELU())
    
    model.add(Convolution2D(64, 3, 3, subsample=(1, 1), init='normal'))
    model.add(ELU())
    
    model.add(Flatten())
    model.add(ELU())
    
    model.add(Dense(100, name='Dense1', init='normal'))
    model.add(ELU())
    
    model.add(Dense(50, name='Dense2', init='normal'))
    model.add(Dropout(0.2))
    model.add(ELU())
    
    model.add(Dense(10, name='Dense3', init='normal'))
    model.add(ELU())
    
    model.add(Dense(1, name='out'))
    
    adam = Adam(lr=1e-6)
    model.compile(optimizer="adam", loss="mse")
    return model

In [8]:
def save_parameters(model):
    model.save_weights('model.h5')
    json_file = open('model.json', mode='w')
    json.dump(model.to_json(), json_file)

In [9]:
model = the_model()
epochs = 40
model.fit_generator(generator(X_train_paths, y_train), 
                    samples_per_epoch= 3*X_train_paths.shape[0],
                    nb_epoch=epochs,
                    verbose=1,
                    validation_data=generator(X_valid_paths, y_valid),
                    nb_val_samples=3*X_valid_paths.shape[0])

Epoch 1/40
19284/19284 [==============================] - 18s - loss: 0.0142 - val_loss: 0.0125
Epoch 2/40
19284/19284 [==============================] - 16s - loss: 0.0109 - val_loss: 0.0115
Epoch 3/40
19284/19284 [==============================] - 16s - loss: 0.0101 - val_loss: 0.0111
Epoch 4/40
19284/19284 [==============================] - 16s - loss: 0.0098 - val_loss: 0.0109
Epoch 5/40
19284/19284 [==============================] - 16s - loss: 0.0095 - val_loss: 0.0104
Epoch 6/40
19284/19284 [==============================] - 16s - loss: 0.0092 - val_loss: 0.0104
Epoch 7/40
19284/19284 [==============================] - 16s - loss: 0.0091 - val_loss: 0.0102
Epoch 8/40
19284/19284 [==============================] - 16s - loss: 0.0089 - val_loss: 0.0099
Epoch 9/40
19284/19284 [==============================] - 16s - loss: 0.0086 - val_loss: 0.0103
Epoch 10/40
19284/19284 [==============================] - 16s - loss: 0.0085 - val_loss: 0.0103
Epoch 11/40
19284/19284 [==============

In [10]:
save_parameters(model)